In [1]:
##imports from libraries
import pandas as pd
import numpy as np
import time
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import resource
import os

In [24]:
## Preprocessing of data
# Load data here:
# get data function
def get_data(data_folder = '/Users/wzh/Downloads/EP3260-MLoNs-2020/dataset/ghg_data/'):
    data = []
    filelist = os.listdir(data_folder)
    for file in filelist:
#         print(file)
        data_single = np.genfromtxt(data_folder+file,dtype=np.float)
        data.append(data_single)
#     data=np.genfromtxt('./ghg_data/ghg.gid.site2067.dat',dtype=np.float)
    return data

def splitDataset(totaldata, train = 0.8, seed = 123, normalize = True):
    # seed
    np.random.seed(seed)
    # number
    numdata = totaldata.shape[0]
    numtrain = int(numdata*train)
    numtest = numdata - numtrain
    # index
    index = np.arange(numdata)
    np.random.shuffle(index)
    # shuffle
    totaldata = totaldata[index,:,:]
    # split
    traindata = totaldata[:numtrain, :, :]
    testdata = totaldata[numtrain:, :, :]
    # split X, Y
    # train
    X_train = traindata[:,:15, :]
    X_train = X_train.reshape(numtrain, -1).T
    Y_train = traindata[:,15, :].T
#     Y_train = Y_train.reshape(numtrain, 1, -1)
    # test
    X_test = testdata[:,:15, :]
    X_test = X_test.reshape(numtest, -1).T
    Y_test = testdata[:,15, :].T
    
    # normalization
    if normalize is True:
        X_train = X_train/np.linalg.norm(X_train, axis=0)
#         Y_train = Y_train/np.linalg.norm(Y_train, axis=0)
        X_test = X_test/np.linalg.norm(X_test, axis=0)
#         Y_test = Y_test/np.linalg.norm(Y_test, axis=0)
    
#     Y_test = Y_test.reshape(numtest, 1, -1)
    return X_train, Y_train, X_test, Y_test
# Split train and test data here: (X_train, Y_train, X_test, Y_test)
data = np.array(get_data())
# Split train and test data here: (X_train, Y_train, X_test, Y_test)
X_train, Y_train, X_test, Y_test = splitDataset(data)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(4905, 2336)
(327, 2336)
(4905, 585)
(327, 585)


In [25]:
## Logistic ridge regression with different optimizers
# cost function and gradient calculation

def cost(x,y,w,lambda_ = 0.01):
    D, N = x.shape
    value = 0
    for i in range(N):
        Z = -1 * y[:,i] * np.dot(w.T , (x[:,i].reshape(D,1)).reshape(D,1))
        value += np.sum(np.log(1+np.exp(Z))) / y.shape[0]
    norm_w = np.linalg.norm(w)
    c = lambda_ * norm_w ** 2
    print("=========================")
    print("loss part 1: {}".format(value/N))
    print("loss part 2, lambda term : {}".format(c))
    return value/N + c 

def function_gradient(X, Y, w, lambda_):
    # Calculate the gradient here:
    # X: DxN
    # Y: dxN
    # w: Dxd
    D, N = X.shape
    d, _ = Y.shape
    
    dw = np.zeros((D, d))
    
    for i in range(N):
        for j in range(d):
            Z = Y[j,i] * np.dot(w[:,j].reshape(1,-1) , (X[:,i].reshape(D,1)).reshape(D,1)) # 1 x 1
            
            dw[:,j] += (X[:,i].reshape(D,1) * Y[j,i]/(1 + np.exp(Z))).reshape(-1) # D x 1 
    c = lambda_ * w *2
    return c-dw/N

In [26]:
## Define solvers: GD, SGD, SVRG and SAG. 
# Setting the values here:

alpha = 0.1 # change the value
num_iters = 5 # change the value
lambda_ = 0.0001 # change the value
epsilon = 0.001 # change the value

# ---------------------- Complete the blank definitions: --------------------------------------

def solver(x,y, w, alpha, num_iters , lambda_ , epsilon , optimizer = "GD",mem=False):
    if (optimizer == "GD") :
        print(num_iters)
        for i in range(num_iters):
            print("hello")
            # update the parameter w for GD here:
            g = function_gradient(x, y, w, lambda_)
            w = w - alpha*g
            loss = cost(x,y,w,lambda_)
            print(i, loss)
            
            if (i%10==0) and (mem):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for GD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
            if (np.linalg.norm(g) <= epsilon):
                break
    elif (optimizer == "SGD"):
        for i in range(num_iters):
            # Complete SGD here:
                pass
    elif (optimizer == "SVRG"):
        T = 100
        K = math.floor(num_iters/T)
        Z = np.matmul(x,np.diagflat(y))
        N = x.shape[1]
        for k in range(K):
            wz = np.matmul(w.T , Z)
            diag = np.diagflat(1/(1+np.exp(-1*wz))-np.ones((1,N)))
            Ga_ = np.matmul(Z , diag)
            ga_ = (1/N) * np.matmul(Ga_ , np.ones((N,1)))
            for t in range(T):
                # Complete SVRG here:
                pass
    elif (optimizer == "SAG"):
        # Complete SAG here:
        pass
    return w

In [27]:
## Solving the optimization problem:

alpha = 0.1 # change the value
num_iters = 5 # change the value
lambda_ = 0.0001 # change the value
epsilon = 0.001 # change the value

y = Y_train
x = X_train
D,N = x.shape
d,_ = y.shape

# w = np.random.rand(D,1)*0.01  # Initialization of w
w = np.random.normal(0,0.1, D*d).reshape(D,d)
start = time.time()
#-------------------- GD Solver -----------------------
num_iters = 5 # change the value
loss = cost(x,y,w,lambda_)
print("initial loss {}".format(loss))

gde = solver(x,y, w, alpha, num_iters , lambda_ , epsilon , optimizer = "GD",mem=False) # complete the command 

end = time.time()
# print("Weights of GD after convergence: \n",gde)

cost_value = cost(x,y,gde,lambda_)  # Calculate the cost value
print("Cost of GD after convergence: ",cost_value)

print("Training time for GD: ", end-start)

#-------------------- SGD Solver -----------------------
# complete here :

#-------------------- SVRG Solver -----------------------
# complete here :

#-------------------- SAG Solver -----------------------
# complete here :

loss part 1: 648.2212300019851
loss part 2, lambda term : 1.6031159829466946
initial loss 649.8243459849318
5
hello
loss part 1: 23.59700226705947
loss part 2, lambda term : 1.6549306335762126
0 25.251932900635683
hello


/Users/wzh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in exp


loss part 1: 22.28208819749881
loss part 2, lambda term : 1.655203666460689
1 23.937291863959498
hello
loss part 1: 21.443807726321005
loss part 2, lambda term : 1.6554737241311346
2 23.09928145045214
hello
loss part 1: 20.824990852055944
loss part 2, lambda term : 1.6557401305225485
3 22.480730982578493
hello
loss part 1: 20.335283291654726
loss part 2, lambda term : 1.6560029244047667
4 21.991286216059493
loss part 1: 648.2212300019851
loss part 2, lambda term : 1.6031159829466946
Cost of GD after convergence:  649.8243459849318
Training time for GD:  195.14756202697754


In [28]:
## Executing the iterations and plot the cost function here:

ti= np.zeros((50,4))
cost_= np.zeros((50,4))
for i in range(50):
    print("......",i,".......")
    #--------------GD-------------------
    start = time.time()
    gde = solver(x.T,y,w,num_iters=i)
    end = time.time()

    cost_[i,0] = cost(x.T,y,gde)

    ti[i,0] = end-start

    #---------------SGD------------------
    #Complete for SGD solver here :
    
    #---------------SVRG----------------
    #Complete for SVRG solver here :
    
    #---------------SAG------------------
    #Complete for SAG solver here :
    
    #------------------------------------
    
    ## Pl the results:
    

l0 = plt.plot(cost_[:,0],color="red")
# complete other plots here: 


plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

l0 = plt.plot(ti[:,0],color="red")
# complete other plots here:

plt.xlabel("Number of Iteration")
plt.ylabel("Time (sec)")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()
    
    
    

...... 0 .......


TypeError: solver() missing 3 required positional arguments: 'alpha', 'lambda_', and 'epsilon'

In [ ]:
## Tunning the hyper-paramter here:

In [ ]:
## Comparing different optimizers here: 